https://gd.aragon.es/ASSOC/asociaciones.html

Tengo que hacer 3 scrapers:
* https://gd.aragon.es/ASSOC/asoc_huesca.html
* https://gd.aragon.es/ASSOC/asoc_zaragoza.html
* https://gd.aragon.es/ASSOC/asoc_teruel.html

In [1]:
from selenium import webdriver
import time

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import urllib.request
from PIL import Image

import pandas as pd
import re

path= "C:\Program Files (x86)\chrimedriver.exe"

# HUESCA

In [2]:
# Abrir una instancia del navegador
driver = webdriver.Chrome(path)
# Va a la página que le digo
driver.get("https://gd.aragon.es/ASSOC/asoc_huesca.html")

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_21088\2303107478.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [3]:
# Localiza el menú desplegable y selecciona la opción deseada
desplegable = Select(driver.find_element("xpath","//*[@id='FIN-C']"))
desplegable.select_by_visible_text('Naturalistas')

# Haz clic en el botón de buscar
buscar_boton = driver.find_element("xpath","//*[@id='avanzada']/div[7]/input[1]")
buscar_boton.click()


In [4]:
data_huesca = []

while True:
    driver.refresh()

    # Obtiene los elementos de la página actual (es una lista por página)
    elementos = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, "//*[@class= 'parrafo2']")))
    
    i = 0
    while i < len(elementos):
        
        elemento = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, "//*[@class= 'parrafo2']")))[i]
        
        xpath_nombre="//*[@id='EdicionPubliBOA']/div/p["+ str(i+2) +"]/span[2]"
        xpath_domicilio= "//*[@id='EdicionPubliBOA']/div/p["+ str(i+2) +"]/span[4]"
        xpath_localidad= "//*[@id='EdicionPubliBOA']/div/p["+ str(i+2) +"]/span[6]"
        
        nombre = elemento.find_element("xpath", xpath_nombre).text
        domicilio= elemento.find_element("xpath", xpath_domicilio).text 
        
        # Hay campos que no tienen localidad
        try:
            localidad= elemento.find_element("xpath", xpath_localidad).text 
        
        except NoSuchElementException:
            localidad= None
          
        # Agregar la información a la lista
        data_huesca.append([nombre, domicilio, localidad])
            
        i += 1
        
    try:
        # Saco los numeritos del texto de paginación
        texto= driver.find_element("xpath","//*[@id='EdicionPubliBOA']/div/h2/span").text
        lista_numeros= re.findall(r'\d+', texto)

        # Si el número del final solo aparece una vez quiere decir que no se ha llegado al final de todo
        if len(set(lista_numeros)) == len(lista_numeros):
            
            # Busca el botón de "siguiente"
            siguiente = driver.find_element("xpath", "//*[@id='EdicionPubliBOA']/div/h2/span/a")

            driver.execute_script("arguments[0].click();", siguiente)
            print("---------SIGUIENTE PÁGINA---------")
            
        else:
            print("---------SE ACABÓ---------")
            break
    except NoSuchElementException:
        print("---------SE ACABÓ---------")
        # El ciclo termina
        break

---------SIGUIENTE PÁGINA---------
---------SE ACABÓ---------


In [5]:
df_huesca = pd.DataFrame(data_huesca, columns=['Nombre', 'Domicilio', 'Localidad'])

In [6]:
df_huesca

,Nombre,Domicilio,Localidad
0,"""VALORES DEL BOSQUE""",PASAJE PINTORA FRIDA KAHLO 12-6D,HUESCA
1,ACTUA PERMACULTURA,Camino Santa Lucía,AYERBE 22800
2,ANUARIO ORNITOLÓGICO DE ARAGÓN - ROCÍN,"C/ PERENA, Nº 30-2º DCHA.",HUESCA
3,"ASOCIACION ""AULA NATURALEZA Y PATRIMONIO CINCA...","C/ Cabañera, 37",22400 MONZON
4,"ASOCIACIÓN ""LA ABEJA MAÑA""","C/ UNICA, Nº 16",SUILS
5,"ASOCIACION ""SOCIEDAD VALDERRASAL""","C/ Barrio Medio, s/n",LOSANGLIS
6,"ASOCIACIÓN ""TIERRA PADMAYATI""","TORRE DE LLEYDA, S/ Nº DE MONZÓN",MONZÓN
7,ASOCIACIÓN ALQUIMIA Y KANELA,"C/ LAS CORTES, 10-1º E",HUESCA
8,ASOCIACION AMIGOS DE LA SELVA,"PL. MAYOR, 6 CASA CHACAS",DENUY (LAS PAULES)
9,ASOCIACIÓN AMIGOS DEL PARQUE GEOLÓGICO Y MINER...,"PLAZA MAYOR, 6",22560 ALCAMPELL


# ZARAGOZA

In [7]:
# Abrir una instancia del navegador
driver = webdriver.Chrome(path)
# Va a la página que le digo
driver.get("https://gd.aragon.es/ASSOC/asoc_zaragoza.html")

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_21088\2224207939.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [8]:
# Localiza el menú desplegable y selecciona la opción deseada
desplegable = Select(driver.find_element("xpath","//*[@id='FIN-C']"))
desplegable.select_by_visible_text('Naturalistas')

# Haz clic en el botón de buscar
buscar_boton = driver.find_element("xpath","//*[@id='avanzada']/div[7]/input[1]")
buscar_boton.click()

In [9]:
data_zaragoza = []

while True:
    driver.refresh()

    # Obtiene los elementos de la página actual (es una lista por página)
    elementos = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, "//*[@class= 'parrafo2']")))
    
    i = 0
    while i < len(elementos):
        
        elemento = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, "//*[@class= 'parrafo2']")))[i]
        
        xpath_nombre="//*[@id='EdicionPubliBOA']/div/p["+ str(i+2) +"]/span[2]"
        xpath_domicilio= "//*[@id='EdicionPubliBOA']/div/p["+ str(i+2) +"]/span[4]"
        xpath_localidad= "//*[@id='EdicionPubliBOA']/div/p["+ str(i+2) +"]/span[6]"
        
        nombre = elemento.find_element("xpath", xpath_nombre).text
        domicilio= elemento.find_element("xpath", xpath_domicilio).text 
        
        # Hay campos que no tienen localidad
        try:
            localidad= elemento.find_element("xpath", xpath_localidad).text 
        
        except NoSuchElementException:
            localidad= None
          
        # Agregar la información a la lista
        data_zaragoza.append([nombre, domicilio, localidad])
            
        i += 1
            
 
    try:
        #siguiente = driver.find_element("xpath", "//*[@id='EdicionPubliBOA']/div/h2/span/a")
        #driver.execute_script("arguments[0].click();", siguiente)
        #print("---------SIGUIENTE PÁGINA---------")
        
        # Saco los numeritos del texto de paginación
        texto= driver.find_element("xpath","//*[@id='EdicionPubliBOA']/div/h2/span").text
        lista_numeros= re.findall(r'\d+', texto)

        # Si el número del final solo aparece una vez quiere decir que no se ha llegado al final de todo
        if len(set(lista_numeros)) == len(lista_numeros):

            # Busca el botón de "siguiente"
            try:
                siguiente = driver.find_element("xpath", "//*[@id='EdicionPubliBOA']/div/h2/span/a[2]")

            except NoSuchElementException:
                siguiente = driver.find_element("xpath", "//*[@id='EdicionPubliBOA']/div/h2/span/a")

            driver.execute_script("arguments[0].click();", siguiente)

            print("---------SIGUIENTE PÁGINA---------")

        else:
            print("---------SE ACABÓ---------")
            break

    except NoSuchElementException:
        print("---------SE ACABÓ---------")
        # El ciclo termina
        break


---------SIGUIENTE PÁGINA---------
---------SIGUIENTE PÁGINA---------
---------SIGUIENTE PÁGINA---------
---------SIGUIENTE PÁGINA---------
---------SIGUIENTE PÁGINA---------
---------SIGUIENTE PÁGINA---------
---------SE ACABÓ---------


In [10]:
df_zaragoza = pd.DataFrame(data_zaragoza, columns=['Nombre', 'Domicilio', 'Localidad'])

In [11]:
df_zaragoza

,Nombre,Domicilio,Localidad
0,"""AETUS"" (ASOCIACION PARA EL ESTUDIO Y CONSERVA...","C/ MARGARITA XIRGU 3, 3-A",50018 ZARAGOZA
1,ADEA ASOCIACION PARA LA DEFENSA Y EL ESTUDIO D...,"C/ PREDICADORES 62, 1-IZDA.",50003 ZARAGOZA
2,AGRUPACION ORNITOLOGICA DE ZARAGOZA (A O Z ),"C/ SAN PABLO, 17",50003 ZARAGOZA
3,AGRUPACION SILVESTRISTA ORNITOLOGICA ARAGONESA...,"FRAY JUAN REGLA, 36",ZARAGOZA
4,AMNISTIA ANIMAL,"PASEO COLON, 6 -LOCALES BAJOS-",50006 ZARAGOZA
...,...,...,...
177,SOCIEDAD ENTOMOLOGICA ARAGONESA,"C/ COSO 95, 3-DCHA",50001 ZARAGOZA
178,"SOCIEDAD ESPAÑOLA DE ORNITOLOGIA SEO, DE POZUE...","C/ RIOJA, 33 (ESTACION DELICIAS)",50011 ZARAGOZA
179,SOCIEDAD MICOLOGICA MONCAYO,"C/ ESCORIAL, 2",50580 VERA DE MONCAYO (ZARAGOZA)
180,"SOCIEDAD VETERINARIA DE ZOOTECNIA, DE MADRID",FACULTAD DE VETERINARIA,ZARAGOZA


# TERUEL

In [12]:
# Abrir una instancia del navegador
driver = webdriver.Chrome(path)
# Va a la página que le digo
driver.get("https://gd.aragon.es/ASSOC/asoc_teruel.html")

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_21088\302263469.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [13]:
# Localiza el menú desplegable y selecciona la opción deseada
desplegable = Select(driver.find_element("xpath","//*[@id='FIN-C']"))
desplegable.select_by_visible_text('Naturalistas')

# Haz clic en el botón de buscar
buscar_boton = driver.find_element("xpath","//*[@id='avanzada']/div[7]/input[1]")
buscar_boton.click()

In [14]:
data_teruel = []

while True:
    driver.refresh()

    # Obtiene los elementos de la página actual (es una lista por página)
    elementos = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, "//*[@class= 'parrafo2']")))
    
    i = 0
    while i < len(elementos):
        
        elemento = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, "//*[@class= 'parrafo2']")))[i]
        
        xpath_nombre="//*[@id='EdicionPubliBOA']/div/p["+ str(i+2) +"]/span[2]"
        xpath_domicilio= "//*[@id='EdicionPubliBOA']/div/p["+ str(i+2) +"]/span[4]"
        xpath_localidad= "//*[@id='EdicionPubliBOA']/div/p["+ str(i+2) +"]/span[6]"
        
        nombre = elemento.find_element("xpath", xpath_nombre).text
        domicilio= elemento.find_element("xpath", xpath_domicilio).text 
        
        # Hay campos que no tienen localidad
        try:
            localidad= elemento.find_element("xpath", xpath_localidad).text 
        
        except NoSuchElementException:
            localidad= None
          
        # Agregar la información a la lista
        data_teruel.append([nombre, domicilio, localidad])
            
        i += 1
        
    try:
        # Saco los numeritos del texto de paginación
        texto= driver.find_element("xpath","//*[@id='EdicionPubliBOA']/div/h2/span").text
        lista_numeros= re.findall(r'\d+', texto)

        # Si el número del final solo aparece una vez quiere decir que no se ha llegado al final de todo
        if len(set(lista_numeros)) == len(lista_numeros):
            
            # Busca el botón de "siguiente"
            siguiente = driver.find_element("xpath", "//*[@id='EdicionPubliBOA']/div/h2/span/a")

            driver.execute_script("arguments[0].click();", siguiente)
            print("---------SIGUIENTE PÁGINA---------")
            
        else:
            print("---------SE ACABÓ---------")
            break
    except NoSuchElementException:
        print("---------SE ACABÓ---------")
        # El ciclo termina
        break

---------SE ACABÓ---------


In [15]:
df_teruel = pd.DataFrame(data_teruel, columns=['Nombre', 'Domicilio', 'Localidad'])

In [16]:
df_teruel

,Nombre,Domicilio,Localidad
0,"""ASOCIACION CULTURAL RINCON VERDE (ACRIVE)""","C/ SAN FRANCISCO, 43..TFNO:",44540 - ALBALATE DEL ARZOBISPO (TERUEL)
1,"""SPA-TERUEL"" SOCIEDAD PROTECTORA DE ANIMALES D...","CRTA. CASTRALVO, 5",TERUEL
2,"ASOCIACIÓN ""EXPLORADOR DE PROXIMIDAD""","CALLE BARÓN DE LA LINDE, Nº 47 - 1º",44550 - ALCORISA (TERUEL)
3,"ASOCIACION ""FLORES DE ANJANA""",C/ LA MUELA S/N,44190 - VILLASPESA (TERUEL)
4,"ASOCIACION ""FUERZA AMBIENTAL""",C/. Carramolino,44210 - CUTANDA (TERUEL)
5,"ASOCIACION ""LA CRUJIENTE ARAGONESA"" DE PERALEJOS","C/ CARASOL, Nº 7",44162 - PERALEJOS (TERUEL)
6,"ASOCIACION ""LA SABINA""","C/San Cristóbal, 10",CALAMOCHA
7,"ASOCIACION ""PEÑA EBOLA""",CALLE NORTE Nº 63,44230 TORNOS
8,"ASOCIACIÓN ""PROYECTO GATO MORA DE RUBIELOS""","URBANIZACIÓN LA PINADA, 31",MORA DE RUBIELOS
9,"ASOCIACION ""PROYECTO GATO ALCAÑIZ""","CALLE MOSEN VICENTE BARDAVIU Nº 2, 1º A",44600 ALCAÑIZ


CREACIÓN DE UNA COLUMNA DE MUNICIPIO PARA DIFERENCIAR ENTRE LOS 3 CONJUNTOS DE DATOS:

In [17]:
df_huesca['Municipio'] = 'Huesca'
df_zaragoza['Municipio'] = 'Zaragoza'
df_teruel['Municipio'] = 'Teruel'

In [18]:
df = pd.concat([df_huesca, df_zaragoza, df_teruel])
df = df.reset_index(drop=True)

In [19]:
# Guardar DataFrame en un archivo de Excel
df.to_excel('scraper_aragon.xlsx', index=False)